In [1]:
import os
import sys

from datetime import datetime, timedelta

import pandas as pd
import matplotlib.pyplot as plt

from pysteps import io, rcparams
from pysteps.utils import conversion, dimension, transformation, clip_domain
from pysteps.visualization import plot_precip_field

# Orkans path
path = "C:/Users/davis.pazars/Documents/orkans"
sys.path.append(path)

from orkans import events


Pysteps configuration file found at: c:\Users\davis.pazars\AppData\Local\miniconda3\envs\nwc-test3\lib\site-packages\pysteps\pystepsrc



In [2]:
date_str = "2022-09-01"
hour_str = "21:00"
n_forw_steps = 8

domain = (2.284e6, 2.622e6, -1.955e6, -1.664e6)

data_src_key = "opera_meteo_france"

img_dir = "C:/Users/davis.pazars/Desktop/nokrisni"


In [3]:
date = datetime.strptime(f"{date_str} {hour_str}", "%Y-%m-%d %H:%M")
# date = datetime.strptime("202205050245", "%Y%m%d%H%M")

data_source = rcparams.data_sources[data_src_key]
root_path = data_source["root_path"]
path_fmt = data_source["path_fmt"]
fn_pattern = data_source["fn_pattern"]
fn_ext = data_source["fn_ext"]
importer_name = data_source["importer"]
importer_kwargs = data_source["importer_kwargs"]
timestep = data_source["timestep"]


# Find the input files from the archive
fns = io.archive.find_by_date(
    date,
    root_path,
    path_fmt,
    fn_pattern,
    fn_ext,
    timestep,
    num_next_files=n_forw_steps,
)

# Read the radar composites
importer = io.get_method(importer_name, "importer")
# read data, quality rasters, metadata
R, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# Convert reflectivity to rain rate
rainrate, metadata = conversion.to_rainrate(R, metadata)


In [7]:
%matplotlib qt
R.shape
haha, hahadata = clip_domain(R, metadata=metadata, extent=(2.192e6, 3.082e6, -2.269e6, -1.278e6))
haha.shape
plot_precip_field(haha[-1, :, :], geodata=hahadata)
# plot_precip_field(R[-1, :, :], geodata=metadata)

<GeoAxes: >

In [ ]:
date_str = "2021-01-05"
date = datetime.strptime(f"{date_str} 00:00", "%Y-%m-%d %H:%M")
tstep = timedelta(minutes=15)
max_iter = 100  # 24 hours * 4 images/hour = 96 images
current_iter = 0

results = pd.DataFrame(columns=["datetime", "region_id", "max_rrate"])

while date.day == 5 and current_iter < max_iter:

    res = events.process_event(date, data_src_key)
    res["datetime"] = pd.to_datetime(res["datetime"])
    res["datetime"] = res["datetime"].dt.floor("h")

    results = pd.concat([results, res])

    date += tstep
    current_iter += 1


In [ ]:
results.groupby(["region_id", "datetime"]).sum("max_rrate").sort_values(
    by=["region_id", "max_rrate"], ascending=[True, False]
).loc[3, :].head(5)


In [ ]:
# Clip domain to include only Baltic states
R, metadata = clip_domain(R, metadata, extent=domain)


In [ ]:
# %matplotlib qt
# plot_precip_field(rainrate[-1, :, :], geodata=metadata)

pic_dir = f'{img_dir}/{date_str}'
if not os.path.exists(pic_dir):
    os.mkdir(pic_dir)
    
date_for_fname = date.strftime("%Y%m%d_%H%M")

%matplotlib  inline
for tstep in range(R.shape[0]):
    plt.figure()
    plot_precip_field(R[tstep, :, :], geodata=metadata, axis='off', colorbar=False)
    plt.title(f'RADAR: {date_str} {hour_str} + {int(15*tstep)}min')
    fname = f"radar_{date_for_fname}_T{int(tstep*15)}.png"
    fname = f'{pic_dir}/{fname}'
    plt.savefig(fname)
    plt.gcf()

In [ ]:
for tstep in range(R.shape[0]):
    print(R[tstep, :, :][R[tstep, :, :] > 1])
    print("")

# Basically, there is just very little data above 1mm/hr, that's why the difference
